In [1]:
!pip install beautifulsoup4==4.12.3 certifi==2024.2.2 charset-normalizer==3.3.2 et-xmlfile==1.1.0 idna==3.6 lxml==5.1.0 numpy==1.26.4 openpyxl==3.1.2 pandas==2.2.0 python-dateutil==2.8.2 pytz==2024.1 requests==2.31.0 six==1.16.0 soupsieve==2.5 tzdata==2024.1 urllib3==2.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.9/120.9 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2024.2
    Uninstalling pytz-2024.2:
      Successfully uninstalled pytz-2024.2
  Attempting uninstall: urllib3
    Found existing install

In [1]:
import pkg_resources

libraries = [
    'beautifulsoup4', 'certifi', 'charset-normalizer', 'et-xmlfile', 'idna',
    'lxml', 'numpy', 'openpyxl', 'pandas', 'python-dateutil', 'pytz',
    'requests', 'six', 'soupsieve', 'tzdata', 'urllib3'
]

for lib in libraries:
    try:
        version = pkg_resources.get_distribution(lib).version
        print(f"{lib}: {version} (instalado correctamente)")
    except pkg_resources.DistributionNotFound:
        print(f"{lib}: No se encontró la librería")


beautifulsoup4: 4.12.3 (instalado correctamente)
certifi: 2024.2.2 (instalado correctamente)
charset-normalizer: 3.3.2 (instalado correctamente)
et-xmlfile: 1.1.0 (instalado correctamente)
idna: 3.6 (instalado correctamente)
lxml: 5.1.0 (instalado correctamente)
numpy: 1.26.4 (instalado correctamente)
openpyxl: 3.1.2 (instalado correctamente)
pandas: 2.2.0 (instalado correctamente)
python-dateutil: 2.8.2 (instalado correctamente)
pytz: 2024.1 (instalado correctamente)
requests: 2.31.0 (instalado correctamente)
six: 1.16.0 (instalado correctamente)
soupsieve: 2.5 (instalado correctamente)
tzdata: 2024.1 (instalado correctamente)
urllib3: 2.2.0 (instalado correctamente)


<ipython-input-1-f16c484b3725>:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import os
import re

def stations():
    link = "https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/"
    response = requests.get(link)
    stn_senamhi = BeautifulSoup(response.text, 'html.parser')

    stn_senamhi2 = re.split(r'nom', str(stn_senamhi))[1:]
    stn = []
    cat = []
    lat = []
    lon = []
    ico = []
    cod = []
    cod_old = []
    estado = []
    data_stn = []

    for i in range(len(stn_senamhi2)):
        x = stn_senamhi2[i].replace('"', '').replace(': ', ":").replace(',\n', "").replace('\\}\{', "")
        data_estaciones = x.split(",")
        stn.append(data_estaciones[0].replace(":", ""))
        cat.append(data_estaciones[1].replace("cate:", ""))
        lat.append(data_estaciones[2].replace("lat:", ""))
        lon.append(data_estaciones[3].replace("lon:", ""))
        ico.append(data_estaciones[4].replace(" ico:", ""))
        cod.append(data_estaciones[5].replace(" cod:", "") if data_estaciones[5][:5] == " cod:" else None)
        cod_old.append(data_estaciones[6].replace("cod_old:", "") if data_estaciones[6][:8] == "cod_old:" else None)
        estado_value = data_estaciones[7] if len(data_estaciones) > 7 else data_estaciones[6]
        estado.append(estado_value.replace("}{", "").replace(" estado:", "") if estado_value[:8] == " estado:" else None)

        data_stn.append(pd.DataFrame({
            'estacion': stn[-1],
            'categoria': cat[-1],
            'lat': lat[-1],
            'lon': lon[-1],
            'ico': ico[-1],
            'cod': cod[-1],
            'cod_old': cod_old[-1],
            'estado': estado[-1]
        }, index=[0]))

    df_stns = pd.concat(data_stn, ignore_index=True)
    return df_stns

def senamhiws_info(code, stations, from_date=None, to_date=None, lat=None, lon=None, stc=None):
    if not code or not isinstance(code, str):
        print("Codigo no definido")
        return None

    df_history_senamhi = []

    if from_date is None and to_date is None:
        from_date = datetime(2016, 1, 1)
        to_date = datetime(2023, 12, 31)
    elif from_date is None and to_date is not None:
        from_date = datetime(2016, 1, 1)
    elif from_date is not None and to_date is None:
        to_date = datetime(2023, 12, 31)

    idx_cod = stations.index[stations['cod'] == code]
    df_idx_stn = stations.loc[idx_cod].reset_index(drop=True)

    ts_date = pd.date_range(from_date, to_date, freq='MS')
    tsw_date = ts_date.strftime('%Y%m')

    for j, date in enumerate(ts_date):
        if pd.isna(df_idx_stn['cod'][0]):
            link = f"https://www.senamhi.gob.pe//mapas/mapa-estaciones-2/export.php?estaciones={df_idx_stn['cod'].iloc[0]}&CBOFiltro={tsw_date[j]}&t_e={df_idx_stn['ico'].iloc[0]}&estado={df_idx_stn['estado'].iloc[0]}&cod_old={df_idx_stn['cod_old'].iloc[0]}"
        else:
            link = f"https://www.senamhi.gob.pe//mapas/mapa-estaciones-2/export.php?estaciones={df_idx_stn['cod'].iloc[0]}&CBOFiltro={tsw_date[j]}&t_e={df_idx_stn['ico'].iloc[0]}&estado={df_idx_stn['estado'].iloc[0]}"

        try:
            print(f"Descargando datos desde: {link}")
            data_stn_senamhi = pd.read_html(link)
        except ValueError as e:
            print(f"Error al leer HTML desde el enlace: {e}")
            print(f"El enlace que falló es: {link}")
            continue

        # Procesar los datos descargados
        data_df_history_senamhi = data_stn_senamhi[1]
        data_df_history_senamhi.columns = data_df_history_senamhi.iloc[0]
        data_df_history_senamhi = data_df_history_senamhi[1:]

        # Eliminar filas que contienen los encabezados repetidos
        data_df_history_senamhi = data_df_history_senamhi[data_df_history_senamhi.iloc[:, 0] != 'AÑO / MES / DÍA']

        # Añadir información adicional
        data_df_history_senamhi["estacion"] = stc
        data_df_history_senamhi["lat"] = lat
        data_df_history_senamhi["lon"] = lon

        # Añadir los datos al historial
        df_history_senamhi.append(data_df_history_senamhi)

    # Combinar todos los datos descargados en un único DataFrame
    if df_history_senamhi:
        df_combined = pd.concat(df_history_senamhi, ignore_index=True)
        output_filename = f"{code}_{df_idx_stn['estacion'].iloc[0]}_historico.xlsx"
        df_combined.to_excel(output_filename, index=False)
        print(f"Datos combinados guardados en: {output_filename}")
    else:
        print("No se encontraron datos para la estación especificada.")

    return df_history_senamhi



#USO DEL CODIGO

In [7]:
# Uso del código
stations_data = stations()
print(stations_data[stations_data["estacion"] == 'ANANEA'])

    estacion categoria        lat        lon ico     cod cod_old estado
566   ANANEA        CO   -14.6763   -69.5339   M  114050    None   REAL


In [8]:
# Filtrar datos de la estación específica
stations_data = stations()

stations_data = stations_data[stations_data["cod"] == '4728C78C']
print("aaaaa", stations_data.head())

aaaaa        estacion categoria         lat     lon ico       cod cod_old  \
703  CUSICANCHA       EMA   -13.48333   -75.3   M  4728C78C    None   

         estado  
703  AUTOMATICA  


In [9]:
#Este codigo descarga los datos de la estacion definida anteriormente y los combina en un solo archivo excel
# Convertir latitud y longitud a números
stations_data['lat'] = pd.to_numeric(stations_data['lat'], errors='coerce')
stations_data['lon'] = pd.to_numeric(stations_data['lon'], errors='coerce')

lat = stations_data['lat'].tolist()[0]
lon = stations_data['lon'].tolist()[0]
stc = stations_data['estacion'].tolist()[0]
codigo = stations_data['cod'].tolist()[0]
print('codigo', codigo)

print("values", lat, lon, stc)

# Llamar a la función para obtener datos históricos
try:
    resultados = senamhiws_info(codigo, stations_data, '2015-01-01', '2024-09-30', lat, lon, stc)
except Exception as e:
    print(f"An error occurred: {e}")

codigo 4728C78C
values -13.48333 -75.3 CUSICANCHA
Descargando datos desde: https://www.senamhi.gob.pe//mapas/mapa-estaciones-2/export.php?estaciones=4728C78C&CBOFiltro=201501&t_e=M&estado=AUTOMATICA
Descargando datos desde: https://www.senamhi.gob.pe//mapas/mapa-estaciones-2/export.php?estaciones=4728C78C&CBOFiltro=201502&t_e=M&estado=AUTOMATICA
Descargando datos desde: https://www.senamhi.gob.pe//mapas/mapa-estaciones-2/export.php?estaciones=4728C78C&CBOFiltro=201503&t_e=M&estado=AUTOMATICA
Descargando datos desde: https://www.senamhi.gob.pe//mapas/mapa-estaciones-2/export.php?estaciones=4728C78C&CBOFiltro=201504&t_e=M&estado=AUTOMATICA
Descargando datos desde: https://www.senamhi.gob.pe//mapas/mapa-estaciones-2/export.php?estaciones=4728C78C&CBOFiltro=201505&t_e=M&estado=AUTOMATICA
Descargando datos desde: https://www.senamhi.gob.pe//mapas/mapa-estaciones-2/export.php?estaciones=4728C78C&CBOFiltro=201506&t_e=M&estado=AUTOMATICA
Descargando datos desde: https://www.senamhi.gob.pe//map